Introduction

The purpose of this code is to build a map in order to plan a benthic sediment sampling survey. THe map will be populated with a number of different elements. These elements will include a base map raster layer in the form of a GeoTIFF showing the hydrographic makeup of the seeabed including a navigational chart and multibeam echosounder derived bathymetry. This will be overlaid with a polygon vector layer, in the form of a shapefile, representing benthic habitat types in the area of interest. Finally a point vector layer will be created from a table of sampling locations. 

Set up/Installation

Here load the necessary modules. The main elements of the map will be tabular data, vector layers and a raster layer. For working with tables we need to load the pandas package along with numpy. In order to create the geometries for the point layer from the coordinates within the table we need shapely. Matplotlib will be necessary for plotting the data. and geopandas will be needed for working with the vector layers and raserio for working with the raster dataset. We will also load a number of dependencies needed for these modules to work such as gdal , fiona, pyproj and glob. I have included an import of os module as well as a line of code to define the path for the Prj.lib file. This contains the information of coordinate reference systems needed for projection of the layers in to the required crs. Without this, the version of python used here would produce an error when loading the raserio package as it cannot find the file on its own. The list of dependencies can be seen below. Here is the URL of the repository containing the code and datasets https://github.com/Rory84/Assignment.git.

In [ ]:
#Set filepath of Proj.lib
import os

os.environ['PROJ_LIB'] = r"C:\Users\Annika\anaconda3\envs\Assignment2\Library\share\proj"
#import modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import shapely.geometry
from shapely.geometry import Point
import gdal
import fiona
import pyproj
import geopandas as gpd
import descartes
import glob
import rasterio
from rasterio.plot import show
from rasterio.crs import CRS

Methodology

The first step is to read the CSV file contining the sampling point information.

In [ ]:
#Read sampling points table and convert to pandas dataframe
'''line 1: read csv to pandas daataframe object named df,
   line 2: select columns from CSV to import
   line 3: print df object'''
df = pd.read_csv("Data\Sampling_Points_coor.csv")
df = df[["FID", "Lat_m", "Lon_m"]]
print(df)

    FID    Lat_m   Lon_m
0     0  6054080  339696
1     1  6052340  341712
2     2  6052010  343485
3     3  6035480  344306
4     4  6033380  346614
5     5  6029520  348000
6     6  6027690  343298
7     7  6033720  344601
8     8  6056230  343370
9     9  6038810  345207
10   10  6029420  347084
11   11  6048190  346080

The code above defines an object "df" using the pd.read_csv method from the pandas package. This reads the CSV file to a pandas DataFrame. The second line defines which columns from the CSV to include in the DataFrame. THe third line prints the object so it can be examined visually.  

The next step defines an object "gdf" using the gpd.GeoDataFrame geopandas method. This takes the df object as the input dataset and, using the gpd.points_from_xy method, selects the columns and coordinate reference system to be used to create GeoSeries object. This is added to the existing non spatial data from the dataframe to return a geopandas GeoDataFrame. The resulting Geodataframe with the additional geometry section is printed below.

In [ ]:
#Convert pandas DataFrame to geopandas GeoDataFrame
'''line 1: Create Geoseries of point geometries from "Lat_m and "Long_m" columns of df object,
   define coordinate reference system as WGS84 UTM Zone 30N using EPSG code'''
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Lon_m, df.Lat_m, crs=32630))
print(gdf)

    FID    Lat_m   Lon_m                        geometry
0     0  6054080  339696  POINT (339696.000 6054080.000)
1     1  6052340  341712  POINT (341712.000 6052340.000)
2     2  6052010  343485  POINT (343485.000 6052010.000)
3     3  6035480  344306  POINT (344306.000 6035480.000)
4     4  6033380  346614  POINT (346614.000 6033380.000)
5     5  6029520  348000  POINT (348000.000 6029520.000)
6     6  6027690  343298  POINT (343298.000 6027690.000)
7     7  6033720  344601  POINT (344601.000 6033720.000)
8     8  6056230  343370  POINT (343370.000 6056230.000)
9     9  6038810  345207  POINT (345207.000 6038810.000)
10   10  6029420  347084  POINT (347084.000 6029420.000)
11   11  6048190  346080  POINT (346080.000 6048190.000)

This code writes the GeoDataFrame to a vector point shapefile called Sampling_Points.shp using the gdf.to_file method. It defines the filepath and specifies the encoding method with the utf-8 unicode commonly used for encoding shapefiles.

In [ ]:
#Write GeoDataFrame to shapefile
'''line1: Write GeoDataFrame to shapefile with geopandas .to_file method,
define encoding method'''
gdf.to_file("Data\Sampling_Points.shp", encoding="utf-8")

Now we load the polygon vector layer called Ards_Map.shp to an object called "Habitat_map". This layer contains a list of benthic habitat types covering the seafloor in our area of interest. In the second line of the code the .to_crs method is used to define the coordinate reference system as the same crs to the point layer we just created.

In [ ]:
#Load habitat map polygon shapefile
'''line 1:Read Ards_Map.shp file to object named Habitat_map,
   line 2:define coordinate reference system of Habitat_map object,
   line 3:print Habitat_map object'''
Habitat_map = gpd.read_file("Data\Ards_Map.shp")
Habitat_map = Habitat_map.to_crs(32630)
print(Habitat_map)

      Rec_Num  ...                                           geometry
0           1  ...  POLYGON ((337105.056 6058468.765, 337129.953 6...
1           2  ...  MULTIPOLYGON (((337105.056 6058468.765, 337129...
2           3  ...  POLYGON ((337064.310 6056844.520, 337089.207 6...
3           4  ...  POLYGON ((337146.001 6057990.217, 337170.898 6...
4           5  ...  POLYGON ((337181.523 6058488.345, 337206.419 6...
...       ...  ...                                                ...
4016     4017  ...  POLYGON ((347738.590 6032454.523, 347740.361 6...
4017     4018  ...  POLYGON ((347632.671 6029908.952, 347657.566 6...
4018     4019  ...  POLYGON ((348517.617 6031798.403, 348542.511 6...
4019     4020  ...  POLYGON ((349222.172 6030446.743, 349247.066 6...
4020     4021  ...  POLYGON ((349444.344 6030756.339, 349469.239 6...

As above, we now load the Sampling_Points.shp we create earlier to an object named "Points". As we defined the CRS when we created the shapefile, there is no need to redefine here. 

In [ ]:
#Load Sampling_Points shapefile
'''line 1: Read Sampling_Points shapefile to object named Points,
   line 2:print Points object'''
Points = gpd.read_file("Data\Sampling_Points.shp")
print(Points)

    FID    Lat_m   Lon_m                        geometry
0     0  6054080  339696  POINT (339696.000 6054080.000)
1     1  6052340  341712  POINT (341712.000 6052340.000)
2     2  6052010  343485  POINT (343485.000 6052010.000)
3     3  6035480  344306  POINT (344306.000 6035480.000)
4     4  6033380  346614  POINT (346614.000 6033380.000)
5     5  6029520  348000  POINT (348000.000 6029520.000)
6     6  6027690  343298  POINT (343298.000 6027690.000)
7     7  6033720  344601  POINT (344601.000 6033720.000)
8     8  6056230  343370  POINT (343370.000 6056230.000)
9     9  6038810  345207  POINT (345207.000 6038810.000)
10   10  6029420  347084  POINT (347084.000 6029420.000)
11   11  6048190  346080  POINT (346080.000 6048190.000)

As you can see the Points object is identical to the GeoDataFrame used to create the shapefile.

THe next line of code creates a link to the raster dataset Ards.tif by reading the file metadat to an object called "chart". This allows the programme to read the file metadata without loading the entire raster dataset. This is helpful when examining large datasets while saving memory.

In [ ]:
#Read raster chart file
chart = rasterio.open("Data\Ards.tif")

Next we can use matplotlib.pyplot to visualise our data.The first two lines of code were taken from https://geobgu.xyz/py/rasterio.html#plotting-rasterio.

In [ ]:
#Plot all layers together
'''line 1:Create a pyplot figure with multiple plots using a single axis
   line 2:pass ax argument to raster layer as "base",
   line 3:plot "Habitat_map" object and pass ax argument, define column and colormap for symbology,
   line 4:plot "Points" object and pass ax argument, define color for symbology
   line 5:show plot'''
fig, ax = plt.subplots()
base = show(chart, ax=ax)
Habitat_map.plot(ax=ax, column="Habitat", cmap='Pastel2');
Points.plot(ax=ax, color="red");
plt.show()

First we create a matplotlib figure and define the subplot axes. As we want to plot each layer on the same plot the plt.subplot argument is left as default meaning there will be a single axis for all plots. The object "base" is created from the "chart" layer with the ax argument defined. The next lines pass the same ax argument to both vector layers in ordeer to ensure all layers are on the same plot. The . plot arguments for the vector layers include symbology definitions with the "habitat_map" object symbolised by the habitat codes contined in the Habitat column.

Expected Results

The code detailed here should produce a map containg the three layers mentioned above. It should have the raster GeoTIFF as a basemap with the habitat map, symbolised by habitat type overlaid. Finally the sampling points should be displayed as the topmost layer, symbolised in red.

Troubleshooting

When I attempted to load the rasterio package I got an error message saying "cannot find proj.db". After a quick internet search I was able to find out that the GDAL dependency needed to have the file path to the coordinate reference system information files defined.

#Set filepath of Proj.lib
import os

os.environ['PROJ_LIB'] = r"C:\Users\Annika\anaconda3\envs\Assignment2\Library\share\proj" 

This section of code was taken from https://stackoverflow.com/questions/56764046/gdal-ogr2ogr-cannot-find-proj-db-error and seemed to solve the issue.

References

https://geobgu.xyz/py/rasterio.html#plotting-rasterio
https://stackoverflow.com/questions/56764046/gdal-ogr2ogr-cannot-find-proj-db-error